# Übung 4: mehr binäre Klassifikation und Challenge

## Aufgabe 1: Titanic

![](images/titanic.jpg)

#### Wir möchten anhand von Daten vorhersagen ob eine Person den Untergang der Titanic überlebt hat. Dafür nehmen wir an einem Challenge auf der Plattform "Kaggle" teil.

1. Registrieren Sie sich auf https://www.kaggle.com/c/titanic und lesen Sie die Einführung auf https://www.kaggle.com/competitions/titanic/overview
2. Machen Sie sich mit den Daten auf https://www.kaggle.com/competitions/titanic/data vertraut
3. Laden Sie die Dateien 
    - Trainingsdaten(train.csv)
    - Testdaten (test.csv)

4. Selektieren Sie sinnvolle Features und teilen Sie die Features in die Kategorien: Numerisch, Ordinal und Nominal auf
3. Bauen Sie jeweils eine Data Pipeline zur Verarbeitung der jeweiligen Features.
4. Benutzen Sie die scitkit-learn Klasse `ColumnTransformer` um die Pipelines zu einer Gesamtpipeline wieder zusammen zu führen.
6. Wenden Sie die Gesamtpipeline auf die Trainingsdaten an 
7. Trainieren Sie mit den Trainingsdaten auf mehreren Klassifiziereren (z.B. `SGDClassifier, DecisionTreeClassifier, SVMClassifier, RandomForestClassifier`) 
8. Evalusieren Sie mit verschiedenen Metriken welcher der beste Klassifizierer ist
9. Schreiben Sie Ihre Ergebnisse in eine CSV Datei. Orientieren Sie sich an dem von Kaggle erwarteten Format `gender_submission.csv`.
10. Senden Sie die Daten ab und sehen Sie wie gut Sie im vergleich zu den anderen Teilnehmenden abschneiden.


In [1]:
import pandas as pd
from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import (
    accuracy_score,
    average_precision_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.svm import SVC

In [2]:
titanic_train = pd.read_csv("data/titanic/train.csv")
titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
ordinal_features = ["Sex"]
nominal_features = ["Embarked"]
numeric_features = ["Pclass", "Age", "SibSp", "Parch", "Fare"]

In [4]:
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())]
)
nominal_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("one_hot_encoding", OneHotEncoder(handle_unknown="ignore")),
    ]
)
ordinal_transfomer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ordinal_encoding", OrdinalEncoder()),
    ]
)

# Gesamtpipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat_nominal", nominal_transformer, nominal_features),
        ("cat_ordinal", ordinal_transfomer, ordinal_features),
    ]
)

set_config(display="diagram")
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']),
                                ('cat_nominal',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('one_hot_encoding',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Embarked']),
                                ('cat_ordinal',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinal_encoding',
                                                  OrdinalEncoder())]),
                                 ['Sex'])])

In [5]:
X = titanic_train.drop("Survived", axis=1)
y = titanic_train[["Survived"]]

X_train, X_test, y_train, y_test = train_test_split(X, y)

y_train_prepared = y_train.copy()  # no pre-processing necessary

In [6]:
X_train_prepared = preprocessor.fit_transform(X_train)
X_train_prepared

array([[ 0.81772738, -0.83096285, -0.49156194, ...,  0.        ,
         1.        ,  1.        ],
       [-1.58862455,  1.1880804 ,  0.46576568, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.81772738, -0.36502979,  0.46576568, ...,  0.        ,
         1.        ,  1.        ],
       ...,
       [ 0.81772738,  0.02324776, -0.49156194, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.81772738, -1.76282896,  3.33774854, ...,  1.        ,
         0.        ,  1.        ],
       [ 0.81772738, -0.90861836, -0.49156194, ...,  0.        ,
         1.        ,  1.        ]])

In [7]:
X_test_prepared = preprocessor.transform(X_test)

classifier = {
    "Neural Network": MLPClassifier(max_iter=1000),
    "Support Vector Machine": SVC(),
    "RandomForest": RandomForestClassifier(),
    "SGD": SGDClassifier(),
    "GradientBoosting": GradientBoostingClassifier(),
}

for name, c in classifier.items():
    print(name)

    clf = c
    clf.fit(X=X_train_prepared, y=y_train_prepared.to_numpy().ravel())
    predicted = clf.predict(X_test_prepared)

    accuracy = accuracy_score(y_pred=predicted, y_true=y_test)
    precision = precision_score(y_pred=predicted, y_true=y_test)
    recall = recall_score(y_pred=predicted, y_true=y_test)
    auc = roc_auc_score(y_true=y_test, y_score=predicted)
    aps = average_precision_score(y_true=y_test, y_score=predicted)
    f1 = f1_score(y_true=y_test, y_pred=predicted)

    print(f"accuracy: {accuracy}")
    print(f"precision: {precision}")
    print(f"recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"AUC: {auc}")
    print("\n")

Neural Network
accuracy: 0.8161434977578476
precision: 0.8028169014084507
recall: 0.6785714285714286
F1 Score: 0.735483870967742
AUC: 0.7889260020554985


Support Vector Machine
accuracy: 0.7937219730941704
precision: 0.7714285714285715
recall: 0.6428571428571429
F1 Score: 0.7012987012987013
AUC: 0.7638746145940392


RandomForest
accuracy: 0.7892376681614349
precision: 0.7605633802816901
recall: 0.6428571428571429
F1 Score: 0.6967741935483871
AUC: 0.7602774922918807


SGD
accuracy: 0.7354260089686099
precision: 0.6582278481012658
recall: 0.6190476190476191
F1 Score: 0.6380368098159509
AUC: 0.7124015073655362


GradientBoosting
accuracy: 0.8161434977578476
precision: 0.8208955223880597
recall: 0.6547619047619048
F1 Score: 0.728476821192053
AUC: 0.7842154847550531




## Load test data and make predictions

In [8]:
titanic_test = pd.read_csv("data/titanic/test.csv")
titanic_test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [9]:
X_test_prepared = preprocessor.transform(titanic_test)
clf = classifier["Support Vector Machine"]
y_pred = clf.predict(X_test_prepared)

In [10]:
submission = titanic_test[["PassengerId"]].copy()
submission["Survived"] = y_pred
submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [11]:
submission.to_csv("data/titanic/submission.csv", index=False)

## Evaluation

Unser einfaches Modell hat eine Accurary Score 0.77272. Das ist sicher kein schlechtes Ergebnis, aber das geht noch deutlich besser. Dinge die wir noch besser machen können:

1. Parameter Tuning: alle Verfahren haben Parameter an denen wir drehen können. Diese Verbesseren die Leistung normal um einige Prozent.
2. Feature Engineering: wir haben die Daten direkt verarbeitet. In dem man geschickt weitere Datenpunkte erzeugt kann die Leistung nochmal deutlich gesteigert werden. z.B. 
    - SibSp und Fare kombiniert werden um den Preis pro Person zu erzeugen. 
    - Oder eine weiter Möglichkeit ist sog. Binning: z.B. kann man das alter in Kleinkind, Kind, Jugendlicher, Erwachsener, Rentner/Pensionär aufteilen. Diese gröbere Unterteilung kann auch ein besseres Ergebnis liefern.
    - Desweiteren kann man aus den Namen Titel wie "Lord, Baron,..." etc extrahieren
    - oder, oder oder
3. Größere Modelle: z.B. mit neuronalen Netzen kann man verschiedene Netz Topologien ausprobieren (dies können wir aktuell noch nicht)
4. Bagging / Boosting (später)